In [23]:
try:
    import unicodecsv
except:
    !pip install unicodecsv
    import unicodecsv

In [2]:
try:
    import numpy
except:
    !pip install numpy
    import numpy

In [3]:
try:
    import nltk
except:
    !pip install nltk
    import nltk

In [4]:
#This will load the csv
import csv
def loadDatasetCsv(filename):
    info = open(filename, "rb")
    has_header = csv.Sniffer().has_header(info.read(1024))
    info.seek(0)
    incsv = csv.reader(info)
    if has_header:
        next(incsv)  #Skip header
    dataset = list(incsv)
    return dataset
filename = 'train_set_x.csv'
dataset_without_classes = loadDatasetCsv(filename)
print('Loaded data file {0} with {1} rows').format(filename, len(dataset_without_classes))
print(dataset_without_classes[18])

Loaded data file train_set_x.csv with 276517 rows
['18', 'Was f\xc3\xbcr eine Mail Was soll da drin stehen']


In [5]:
#This will load the csv
import csv
def loadClassCsv(filename):
    info = open(filename, "rb")
    has_header = csv.Sniffer().has_header(info.read(1024))
    info.seek(0)
    incsv = csv.reader(info)
    if has_header:
        next(incsv)  #Skip header
    dataset = list(incsv)
    return dataset
filename = 'train_set_y.csv'
classds = loadClassCsv(filename)
print('Loaded data file {0} with {1} rows').format(filename, len(classds))
print(classds[18])

Loaded data file train_set_y.csv with 276517 rows
['18', '3']


In [6]:
#get Id from list object
def getId(line):
    info = csv.reader(line, delimiter=',')
    new_line = list(info)
    return new_line[0]

In [7]:
#get Utterance from list object
def getUtt(line):
    info = csv.reader(line, delimiter=',')
    new_line = list(info)
    return new_line[1]
#print(getUtt(dataset[1]))

In [8]:
def getClass(line):
    info = csv.reader(line, delimiter=',')
    new_line = list(info)
    new_line_item = ' '.join(new_line[1])
    return new_line_item

In [9]:
def appendClass(dataset):
    for i in range(len(dataset)):
        dataset[i].append(getClass(classds[i])) #Can assume this since they're all in the same position
    return dataset
dataset_with_classes = appendClass(dataset_without_classes)
print(dataset_with_classes[20])

['20', 'Pas mal pour une ligne Tu sugg\xc3\xa8res que Collard choisit', '1']


In [10]:
#Separate by class
#Assuming last index has the class nb
def separateByClass(dataset):
    separated = {}
    for i in range(len(dataset)):
        vector = dataset[i]
        if (int(vector[-1]) not in separated):
            separated[int(vector[-1])] = []
        separated[int(vector[-1])].append(vector)
    return separated

In [11]:
def createBigrams(ds, n):
    text = getUtt(ds[n])
    vector = ds[n]
    class1 = vector[-1]
    textNeeded = ''.join(text)
    textNeeded = textNeeded.replace(" ","")
    n = 2
    bigrams = [[(textNeeded[i:i+n]),class1] for i in range(0,len(textNeeded), 1)]
    return bigrams

all_ngrams = []
for i in range(len(dataset_with_classes)):
    all_ngrams.extend(createBigrams(dataset_with_classes,i))
print(all_ngrams[34])

separated = separateByClass(all_ngrams)
# for i in range(len(all_ngrams)):
#     if "é" in all_ngrams[i]:
#         print(all_ngrams[i])
#         del all_ngrams[i]

['eq', '1']


In [12]:
def calculatePrior(classDataset, language):
    print("Calculating prior")
    sum = len(classDataset)
    occurence = 0
    for i in range(sum):
        if(getClass(classDataset[i]) == language):
            occurence = occurence + 1
    prior = occurence/float(sum)
    return prior
nb = calculatePrior(classds, '4')
print(nb)

Calculating prior
0.0509154952498


In [13]:
def calculateLikelihood(all_seq, data, class_wanted):
    #print("Calculating likelihood")
    arr = separated[class_wanted]
    occurence = 0
    for i in range(len(separated[class_wanted])):
        if data in arr[i]:
            occurence += 1
    h = occurence/float(len(separated[class_wanted]))
    return h
    
# nb = calculateLikelihood(all_ngrams, "er", 1)
# print(nb)

In [14]:
#This isn't getting used
def classify_a_bigram(bigram):
    arr_of_possibilities = [0,0,0,0,0]
    #Hardcoded the prior calculated above because my computer slow AF
    prior_arr = [0.0512337396977,0.510937844689,0.253054965879,0.133857954484,0.0509154952498]
    arr_of_langs = ["Slovak","French","Spanish","German","Polish"]
    for i in range(len(arr_of_possibilities)):
        #print("Calculation of stats for " + arr_of_langs[i])
        prior = prior_arr[i]
        likelihood = calculateLikelihood(all_ngrams, bigram, i)
        arr_of_possibilities[i] = prior*likelihood
    if arr_of_possibilities == [0,0,0,0,0]:
        lang = 1
    else:
        lang = numpy.argmax(arr_of_possibilities)
    return lang
#print(classify_a_bigram("vp"))

In [22]:
import csv
def loadDatasetCsv(filename):
    info = open(filename, "rb")
    has_header = csv.Sniffer().has_header(info.read(1024))
    info.seek(0)
    incsv = csv.reader(info)
    if has_header:
        next(incsv)  #Skip header
    dataset = list(incsv)
    return dataset
filename = 'test_set_x.csv'
dataset_test_set = loadDatasetCsv(filename)
print('Loaded data file {0} with {1} rows').format(filename, len(dataset_test_set))
#length = len(dataset_test_set)
likelihood = {}
likelihood[0] = []
likelihood[1] = []
likelihood[2] = []
likelihood[3] = []
likelihood[4] = []
#Hardcoded the prior calculated above because my computer slow AF
prior_arr = [0.0512337396977,0.510937844689,0.253054965879,0.133857954484,0.0509154952498]
with open('output.csv', "wb") as csvfile:
    writer = csv.writer(csvfile)
    header = "Id"
    category = "Category"
    writer.writerow([header] + [category])
    for i in range(length):
        ngrams = createBigrams(dataset_test_set,i)
        ngrams = ['je','te','tu','no','us']
        print("We are at the sentence " + str(i) + " on 118508")
        for j in range(len(ngrams)):
            print(likelihood[0])
            likelihood[0].append(calculateLikelihood(all_ngrams, ngrams[j], 0))
            likelihood[1].append(calculateLikelihood(all_ngrams, ngrams[j], 1))
            likelihood[2].append(calculateLikelihood(all_ngrams, ngrams[j], 2))
            likelihood[3].append(calculateLikelihood(all_ngrams, ngrams[j], 3))
            likelihood[4].append(calculateLikelihood(all_ngrams, ngrams[j], 4))
        #array_of_languages.append(classify_a_bigram(ngrams[j]))
        k=0
        array_of_languages = []
        while k < 5:
            prod = numpy.prod(likelihood[k])
            #Bayesian for each language
            array_of_languages.append(prod*prior_arr[k])
            k = k + 1
        #Winning language
        language = str(numpy.argmax(array_of_languages))
        writer.writerow(getId(dataset_test_set[i]) + [language])
        
# ngrams = createBigrams(dataset_test_set,3)
# print(ngrams)
# all_ngrams_with_classes = []
# for i in range(len(dataset_test_set)):
#     all_ngrams_with_classes.extend(createBigrams(dataset_test_set,i))

Loaded data file test_set_x.csv with 118508 rows
We are at the sentence 0 on 118508
[]
[0.007956470243629093]
[0.007956470243629093, 0.008656919804892959]
[0.007956470243629093, 0.008656919804892959, 0.0017367965257701762]
[0.007956470243629093, 0.008656919804892959, 0.0017367965257701762, 0.005103730212299895]
1.70480326411e-12
2.1343833453e-12
4.40503665e-13
1.15590015592e-13
2.64319139504e-13
1
